In [24]:
%reload_ext autoreload
%autoreload 2

from pathlib import Path
import sys 

project_root = Path().cwd()
scripts_dir = project_root / "src" / "scripts"
sys.path.insert(0, str(scripts_dir))

import numpy as np 
import scipy.stats as stats
import scipy.io as io
import os 
import matplotlib.pyplot as plt 
import seaborn as sns
import process_2p as p2p
import lick_behav_analysis as behav

#Recording parameters
fps = 15
framespertrial = 375 
ms_per_frame = int(1000//fps)
reward_frame = 150
cue_frame = 105
trials = 30
cmap= "coolwarm"

In [23]:
p2p

<module 'process_2p' from '/Users/savani/Documents/PangLab/TwoPhotonAnalysis/src/scripts/process_2p.py'>

In [25]:
folder = '/Users/savani/Downloads/2p_data/water deprived 30 sucrose'
all_avg_f, all_baseline_data, all_aligned_f = p2p.process_2p_folder(folder)

(133, 225)